https://tutorials.pytorch.kr/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py

## AUTOGRAD

`autograd` 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공합니다. 이는 실행-기반-정의(define-by-run) 프레임워크로, 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라집니다.

패키지의 중심에는 `torch.Tensor` 클래스가 있습니다. 만약 `.requires_grad` 속성을 `True` 로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작합니다. 계산이 완료된 후 `.backward()` 를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있습니다. 이 Tensor의 변화도는 `.grad` 속성에 누적됩니다.

Tensor가 기록을 추적하는 것을 중단하게 하려면, `.detach()` 를 호출하여 연산 기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지할 수 있습니다.

기록을 추적하는 것(과 메모리를 사용하는 것)을 방지하기 위해, 코드 블럭을 `with torch.no_grad():` 로 감쌀 수 있습니다. 이는 특히 변화도(gradient)는 필요없지만, `requires_grad=True` 가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(evaluate)할 때 유용합니다.

Autograd 구현에서 매우 중요한 클래스가 하나 더 있는데, 이것은 바로 Function 클래스입니다.

Tensor 와 Function 은 서로 연결되어 있으며, 모든 연산 과정을 부호화(encode)하여 순환하지 않는 그래프(acyclic graph)를 생성합니다. 각 tensor는 `.grad_fn` 속성을 갖고 있는데, 이는 Tensor 를 생성한 Function 을 참조하고 있습니다. (단, 사용자가 만든 Tensor는 예외로, 이 때 `grad_fn` 은 `None` 입니다.)

도함수를 계산하기 위해서는 Tensor 의 `.backward()` 를 호출하면 됩니다. 만약 Tensor 가 스칼라(scalar)인 경우(예. 하나의 요소 값만 갖는 등)에는 backward 에 인자를 정해줄 필요가 없습니다. 하지만 여러 개의 요소를 갖고 있을 때는 tensor의 모양을 gradient 의 인자로 지정할 필요가 있습니다.

In [1]:
import torch

In [2]:
x = torch.ones(2, 2, requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [3]:
y = x + 2
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [4]:
y.grad_fn

In [5]:
z = y*y*3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [7]:
a = torch.randn(2, 2)
a = a*3
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = a*a
print(b)

False
True
tensor([[2.8197, 0.9979],
        [1.6427, 1.7477]], grad_fn=<MulBackward0>)


In [10]:
out.backward()
# same as out.backward(torch.tensor(1.))
# out은 하나의 스칼라값만 가지고 있기 때문에

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [11]:
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

4.5 로 이루어진 행렬을 확인할 수 있습니다. out 을 Tensor $o$ 라고 하면, 다음과 같이 구할 수 있습니다. 
$$ o = \frac{1}{4}*\sum_i z_i$$
$$ z_i = 3*(x_i + 2)^2$$
$$ z_i∣_{x_i=1} = 27 $$ 
$$ \frac{\partial{o}}{\partial{x_i}} = \frac{3}{2}*(x_i + 2)$$
$$ \therefore \frac{\partial{o}}{\partial{x_i}}∣_{x_i=1} = \frac{9}{2} = 4.5$$



수학적으로 벡터 함수 $\vec{y} = f(\vec{x})$ 에서 $\vec{x}$에 대한 $\vec{y}$의 변화도는 야코비안 행렬(Jacobian Matrix)입니다

$$ 